In [11]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import gpxpy
from pyproj import Transformer

transformer = Transformer.from_crs("EPSG:4326", "EPSG:2056", always_xy=True)

In [5]:
filepath = "../data/7-gipfel-tour-flumserberg.gpx"

In [ ]:
gdf_waypoints = gpd.read_file(path_v2, layer="waypoints", driver="GPX")

gdf_waypoints = gdf_waypoints.set_crs(epsg=4326).to_crs(epsg=2056)

pts_3d = [Point(x, y, z) for x, y, z in zip(gdf_waypoints.geometry.x, gdf_waypoints.geometry.y, gdf_waypoints["ele"])]

gdf_waypoints = gdf_waypoints.set_geometry(pts_3d, crs=gdf_waypoints.crs)

gdf_waypoints = gdf_waypoints[["swisstopo_waypoint_id", "name", "geometry"]]

gdf_waypoints

,swisstopo_waypoint_id,name,geometry
0,1,Startpunkt,POINT Z (2737832.731 1214786.689 1974.2)
1,2,Ziel,POINT Z (2739850.838 1217402.568 1379.8)


In [ ]:


lines_lv95 = []

with open(path_v2, 'r', encoding='utf-8') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

    for track in gpx.tracks:
        for segment in track.segments:
            # für jeden Segment eine eigene Koordinatenliste
            coords = []
            for pt in segment.points:
                lon, lat, elev = pt.longitude, pt.latitude, pt.elevation or 0.0
                e, n = transformer.transform(lon, lat)
                coords.append((e, n, elev))
            # 3D-LineString erzeugen
            lines_lv95.append(LineString(coords))

# GeoDataFrame erstellen
gdf_lines = gpd.GeoDataFrame(
    { "geometry": lines_lv95 },
    crs="EPSG:2056"   # LV95
)

gdf_lines

,geometry
0,"LINESTRING Z (2737832.731 1214786.689 1974.2, ..."


In [ ]:

## Erstellung Waypoints GeoDataFrame
gdf_waypoints = gpd.read_file(filepath, layer="waypoints", driver="GPX")
gdf_waypoints = gdf_waypoints.set_crs(epsg=4326).to_crs(epsg=2056)
pts_3d = [Point(x, y, z) for x, y, z in zip(gdf_waypoints.geometry.x, gdf_waypoints.geometry.y, gdf_waypoints["ele"])]
gdf_waypoints = gdf_waypoints.set_geometry(pts_3d, crs=gdf_waypoints.crs)[["swisstopo_waypoint_id", "name", "geometry"]]


## Erstellung des Tracks GeoDataFrames
lines_lv95 = []
identifier_segment = 0
identifier_list = []

with open(filepath, 'r', encoding='utf-8') as gpx_file:
    gpx = gpxpy.parse(gpx_file)
    for track in gpx.tracks:
        for segment in track.segments:
            coords = []
            identifier_segment += 1
            for pt in segment.points:
                lon, lat, elev = pt.longitude, pt.latitude, pt.elevation or 0.0
                e, n = transformer.transform(lon, lat)
                coords.append((e, n, elev))
            # 3D-LineString erzeugen
            lines_lv95.append(LineString(coords))
            identifier_list.append(identifier_segment)

gdf_lines = gpd.GeoDataFrame(
    {"id": identifier_list, "geometry": lines_lv95},
    crs="EPSG:2056"   # LV95
)


## Erstellung des Ausgabe GeoDataFrames
gdf_app = gpd.GeoDataFrame(
    {}

In [16]:
gdf_waypoints

,swisstopo_waypoint_id,name,geometry
0,1,Startpunkt,POINT Z (2737832.731 1214786.689 1974.2)
1,2,Ziel,POINT Z (2739850.838 1217402.568 1379.8)


In [17]:
gdf_lines

,id,geometry
0,1,"LINESTRING Z (2737832.731 1214786.689 1974.2, ..."


In [20]:
import geopandas as gpd
from shapely.geometry import Point

def map_line_endpoints_nearest(
    gdf_lines: gpd.GeoDataFrame,
    gdf_waypoints: gpd.GeoDataFrame,
) -> gpd.GeoDataFrame:
    """
    Für jede LineString-Geometrie in gdf_lines sucht diese Funktion
    den nächsten Punkt in gdf_waypoints und schreibt:
      - start_wp_id  (swisstopo_waypoint_id des nächsten Waypoints am Linienanfang)
      - end_wp_id    (… am Linienende)
      - dist_start   (Distanz in m zum Start-Waypoint)
      - dist_end     (Distanz in m zum End-Waypoint)

    Beide GeoDataFrames müssen dasselbe CRS (z.B. EPSG:2056) haben.
    """
    # 2) Start- und Endpunkt-Frames bauen
    starts = gpd.GeoDataFrame(
        {"line_idx": gdf_lines.index},
        geometry=gdf_lines.geometry.apply(lambda ln: Point(ln.coords[0])),
        crs="EPSG:2056"
    )
    ends = gpd.GeoDataFrame(
        {"line_idx": gdf_lines.index},
        geometry=gdf_lines.geometry.apply(lambda ln: Point(ln.coords[-1])),
        crs="EPSG:2056"
    )

    # 3) Nearest-Join durchführen
    joined_s = gpd.sjoin_nearest(
        starts,
        gdf_waypoints[["swisstopo_waypoint_id", "geometry"]],
        how="left",
        distance_col="dist_start",
    )
    joined_e = gpd.sjoin_nearest(
        ends,
        gdf_waypoints[["swisstopo_waypoint_id", "geometry"]],
        how="left",
        distance_col="dist_end",
    )

    # 4) Ergebnisse zurück ins Original-Frame mergen
    #    wir resetten index, damit wir 'line_idx' zum Mergen haben
    result = (
        gdf_lines
        .reset_index(drop=False)  # index → Spalte 'index'
        .merge(
            joined_s[["line_idx", "swisstopo_waypoint_id", "dist_start"]],
            left_on="index", right_on="line_idx",
            how="left"
        )
        .rename(columns={"swisstopo_waypoint_id": "start_wp_id"})
        .merge(
            joined_e[["line_idx", "swisstopo_waypoint_id", "dist_end"]],
            on="line_idx", how="left"
        )
        .rename(columns={"swisstopo_waypoint_id": "end_wp_id"})
        .set_index("index")
        .drop(columns=["line_idx"])
    )

    return result


In [21]:
map_line_endpoints_nearest(
    gdf_lines=gdf_lines,
    gdf_waypoints=gdf_waypoints
).head(3)

,id,geometry,start_wp_id,dist_start,end_wp_id,dist_end
index,,,,,,
0,1,"LINESTRING Z (2737832.731 1214786.689 1974.2, ...",1,0.0,2,0.0
